In [ ]:
#Importing drive model from the google.colab package
from google.colab import drive

#Mounting the google drive to a specific path
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/GitHub_Projects/

/content/drive/MyDrive/GitHub_Projects


In [ ]:
!git config --global user.email "dorothy.sarpongk@gmail.com"
!git config --global user.name "01DorSarpong"

In [ ]:
!git clone https://github.com/01DorSarpong/FTSE_350_Anomaly_Detection.git

Cloning into 'FTSE_350_Anomaly_Detection'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), 7.44 KiB | 1.06 MiB/s, done.


In [ ]:
%cd FTSE_350_Anomaly_Detection/

/content/drive/MyDrive/GitHub_Projects/FTSE_350_Anomaly_Detection


In [8]:
%pwd

'/content/drive/MyDrive/GitHub_Projects/FTSE_350_Anomaly_Detection'

In [9]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Unsupervised_Anomaly_Detection.ipynb

nothing added to commit but untracked files present (use "git add" to track)


In [ ]:
!git add Unsupervised_Anomaly_Detection.ipynb